# COMP5318 - Machine Learning and Data Mining: Assignment 1

In [2]:
import pandas as pd
import os
print(os.listdir("./Input/train"))
pd.set_option('display.max_columns', 10)

from IPython.display import set_matplotlib_formats, display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import graphviz

%matplotlib inline

['train.csv']


In [3]:
# train.csv including feature and label using for training model.
data_train_df = pd.read_csv('./Input/train/train.csv') 

In [4]:
# Selecting input feature
data_train_feature = data_train_df.loc[:, "v1":"v784"].to_numpy()

# Selecting output lable 
data_train_label = data_train_df.label.to_numpy()

In [5]:
# Treain Test Split 
X_train, X_test, y_train, y_test = train_test_split(
    data_train_feature, data_train_label, random_state=0, stratify=data_train_label)

## ACCURACIES BEFORE PRE-PROCESSING

In [11]:
debug=False  
'''
Set this to True if you want to see how the models perform without any pre-processing. 
We used this information in the report to see how much of an improvement pre-processing
will make in terms of accuracy and computation times but we don't need that information 
everytime the code is run. 
'''

In [12]:
%%time
# Default KNN before pre-processing 
if debug:
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    print("Accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

Accuracy on test set: 0.841
CPU times: user 9.55 s, sys: 1.8 s, total: 11.4 s
Wall time: 4.08 s


In [13]:
%%time
# Default LogReg before pre-processing
if debug:
    logreg = LogisticRegression(max_iter = 5000)
    logreg.fit(X_train, y_train)
    print("Accuracy on test set:", logreg.score(X_test, y_test))
    
'''
Since there is no pre-processing, there are too many features and data. Therefore, the limit exceed
error will always be thrown. We will see later how this error gets removed because of pre-processing
'''

Accuracy on test set: 0.7902666666666667
CPU times: user 56min 50s, sys: 3min 6s, total: 59min 57s
Wall time: 10min 7s


/Users/aryanbhatia/opt/anaconda3/envs/comp5318/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
%%time
# Default NB before pre-processing 
if debug:
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    y_pred = nb.predict(X_test)
    print("Accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

Accuracy on test set: 0.601
CPU times: user 855 ms, sys: 215 ms, total: 1.07 s
Wall time: 615 ms


In [15]:
%%time 
# Default DT before pre-processing
if debug:
    tree = DecisionTreeClassifier(criterion='entropy', max_depth=12, random_state=42)
    tree.fit(X_train, y_train)
    print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))

Accuracy on test set: 0.793
CPU times: user 8.36 s, sys: 62 ms, total: 8.42 s
Wall time: 8.48 s


In [16]:
%%time 
# Default Poly SVM before pre-processing 
if debug:
    poly_svm = SVC(kernel="poly", degree=2) #polynomial kernel with degree 2
    poly_svm.fit(X_train, y_train)
    y_pred = poly_svm.predict(X_test)
    print("SVM with polynomial kernel - accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

SVM with polynomial kernel - accuracy on test set: 0.859
CPU times: user 1min 19s, sys: 462 ms, total: 1min 19s
Wall time: 1min 20s


## DATA PRE-PROCESSING FOR TRAINING DATA

In [18]:
# Normalisation
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_norm = scaler.transform(X_train)
X_test_norm  = scaler.transform(X_test)

pd.DataFrame(X_train_norm)

,0,1,2,3,4,...,779,780,781,782,783
0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,...,0.462745,0.38,0.082353,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
22495,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
22496,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
22497,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
22498,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0


In [19]:
# Dimension Reductionality 
pca = PCA(n_components=0.9).fit(X_train_norm)

X_train_pca = pca.transform(X_train_norm)
X_test_pca = pca.transform(X_test_norm)

pd.DataFrame(X_train_pca)

,0,1,2,3,4,...,79,80,81,82,83
0,2.927449,-5.553593,4.019518,-1.395135,1.064630,...,-0.011555,-0.025837,-0.095283,-0.109759,-0.024098
1,1.992421,0.442999,-2.368003,0.811313,-1.177249,...,-0.479448,0.015362,-0.228318,-0.181703,-0.037604
2,2.916436,-4.533643,2.285697,-2.685905,0.292165,...,-0.053102,0.513047,-0.128389,0.073825,-0.079035
3,-6.189816,1.348044,-0.645078,-2.423576,1.313570,...,-0.153498,-0.184887,0.326095,0.042007,-0.248368
4,-2.974141,-4.625361,1.107169,0.583185,0.232016,...,0.265475,0.093921,0.410109,0.028499,0.002914
...,...,...,...,...,...,...,...,...,...,...,...
22495,1.257244,5.409011,5.380687,3.621838,-2.909050,...,-0.069430,0.024253,-0.329398,-0.153290,0.420994
22496,1.754955,-4.031853,0.866494,-0.421730,0.518514,...,-0.398757,-0.208873,0.206416,-0.112603,0.073324
22497,-6.374198,0.561054,-0.780198,-2.838871,1.389735,...,0.245362,0.187905,-0.085312,-0.223798,-0.003016
22498,-5.730576,2.660773,-0.073577,-3.724108,1.928655,...,0.021028,0.012773,0.234765,0.361081,0.209532


## DATA PRE-PROCESSING FOR TESTING DATA

In [20]:
# test_input.csv includes 5000 samples used for label prediction. Test samples do not have labels.
data_test_df = pd.read_csv('./Input/test/test_input.csv', index_col=0) 

In [21]:
# Data Normalisation
output_test_norm = scaler.transform(data_test_df.to_numpy())

pd.DataFrame(output_test_norm)

,0,1,2,3,4,...,779,780,781,782,783
0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
1,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
2,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
3,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4996,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4997,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4998,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0


In [22]:
# Dimension Reduction
output_test_pca = pca.transform(output_test_norm)

pd.DataFrame(output_test_pca)

,0,1,2,3,4,...,79,80,81,82,83
0,2.030060,-5.433903,1.980836,-1.898502,-1.143763,...,0.163122,0.084214,0.142833,0.013530,-0.139753
1,1.971218,-5.661186,1.332592,-1.375247,-0.702272,...,-0.407891,0.289796,-0.219731,-0.249055,-0.450584
2,0.527241,-5.844876,1.605396,-1.962709,-2.800743,...,-0.056665,0.077233,0.170281,0.118583,0.053545
3,-0.334794,-3.352547,-0.780979,1.740940,0.259736,...,-0.145004,-0.078867,-0.272437,-0.101939,0.117087
4,-1.495075,-2.585312,-0.883327,0.401559,0.713086,...,0.370981,0.381618,-0.013033,0.024698,0.149657
...,...,...,...,...,...,...,...,...,...,...,...
4995,2.789104,-3.193621,0.806616,0.593713,-0.233942,...,0.041395,-0.117392,-0.476785,-0.203433,-0.099679
4996,2.489568,5.947833,-0.281650,1.209622,3.791362,...,-0.535142,0.092127,0.591728,0.717596,-0.166519
4997,-5.653882,-0.605367,-1.218067,3.388302,-0.931545,...,-0.497402,-0.183685,-0.146962,0.410792,-0.063161
4998,7.083845,1.511506,0.441606,1.107137,0.528356,...,-0.400921,0.021544,-0.081451,0.188384,-0.141796


## KNN

### After Pre-Processing

In [ ]:
%%time
# Accuracy of default KNN classifier after pre processing 
knn = KNeighborsClassifier()
knn.fit(X_train_pca, y_train)
y_pred = knn.predict(X_test_pca)
print("Accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

### Parameter Tuning

In [ ]:
%%time
# Parameter Tuning 
param_grid = {'n_neighbors': [1, 3, 5, 11, 15], 'p': [1, 2]}
print("Parameter grid:\n{}".format(param_grid))

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, return_train_score=True, n_jobs=-1)

grid_search.fit(X_train_pca, y_train)

print("Test set score: {:.2f}".format(grid_search.score(X_test_pca, y_test)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
%%time
# Create a KNN Classifier using best parameters 
best_n = grid_search.best_params_['n_neighbors']
best_p = grid_search.best_params_['p']
knn = KNeighborsClassifier(n_neighbors=best_n, p=best_p)
knn.fit(X_train_pca, y_train)
y_pred = knn.predict(X_test_pca)
print("\nAccuracy on test set after parameter tuning: {:.3f}".format(accuracy_score(y_test, y_pred)))

### Exporting

In [ ]:
# KNN Predictions
predictions = []

for i in output_test_pca:
    prediction = knn.predict([list(i)])
    predictions.append(prediction[0])
    
# Output to CSV file 
output_df = pd.DataFrame(predictions, columns = ['label'])
output_df.to_csv('./Output/knn.csv', sep=",", float_format='%d', index_label="id")

## LOGISTIC REGRESSION

### After pre-processing 

In [ ]:
%%time
# Accuracy of default LogReg classifier after pre processing 
logreg = LogisticRegression(max_iter = 5000)
logreg.fit(X_train_pca, y_train)
print("Accuracy on test set:", logreg.score(X_test_pca, y_test)*100)

### Parameter Tuning

## NAIVE BAYES

### After pre-processing

In [23]:
%%time
# Accuracy of default NB Classfier after pre processing 
nb = GaussianNB()
nb.fit(X_train_pca, y_train)
y_pred = nb.predict(X_test_pca)
print("Accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

Accuracy on test set: 0.771
CPU times: user 55.6 ms, sys: 15.9 ms, total: 71.5 ms
Wall time: 70 ms


### Parameter Tuning

In [24]:
%%time
# Parameter Tuning 
param_grid_nb = { 'var_smoothing': np.logspace(0,-9, num=100) }

nbModel_grid = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=-1)
nbModel_grid.fit(X_train_pca, y_train)

print("Test set score: ", nbModel_grid.score(X_test_pca, y_test))
print("Best parameters: {}".format(nbModel_grid.best_params_))
print("Best cross-validation score: ", nbModel_grid.best_score_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Test set score:  0.772
Best parameters: {'var_smoothing': 0.0002848035868435802}
Best cross-validation score:  0.7708888888888887
CPU times: user 3.96 s, sys: 875 ms, total: 4.83 s
Wall time: 21.2 s


In [25]:
%%time
# Create a NB Classifier using best parameters and check accuracy 
ideal_var_smoothing = nbModel_grid.best_params_['var_smoothing']
nb = GaussianNB(var_smoothing=ideal_var_smoothing)
nb.fit(X_train_pca, y_train)
y_pred = nb.predict(X_test_pca)
print("\nAccuracy on test set after parameter tuning: {:.3f}".format(accuracy_score(y_test, y_pred)))


Accuracy on test set after parameter tuning: 0.772
CPU times: user 57.4 ms, sys: 15 ms, total: 72.4 ms
Wall time: 70.9 ms


### Exporting

In [26]:
# NB Predictions 
predictions = []

for i in output_test_pca:
    prediction = nb.predict([list(i)])
    predictions.append(prediction[0])

output_df = pd.DataFrame(predictions, columns = ['label'])
output_df.to_csv('./Output/naive.csv', sep=",", float_format='%d', index_label="id")

## DECISION TREE

### After pre-processing

In [ ]:
%%time
# Accuracy of default DT classifier after pre-processing 
tree = DecisionTreeClassifier(criterion='entropy', max_depth=12, random_state=42)
tree.fit(X_train_pca, y_train)
print("Accuracy on test set: {:.3f}".format(tree.score(X_test_pca, y_test)))

### Parameter Tuning 

## SVM

### After pre-processing

In [ ]:
%%time
# SVM with polynomial kernel
poly_svm = SVC(kernel="poly", degree=2) #polynomial kernel with degree 2
poly_svm.fit(X_train_pca, y_train)
y_pred = poly_svm.predict(X_test_pca)
print("SVM with polynomial kernel - accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

In [ ]:
%%time
# SVM with Linear Kernel
lin_svm = SVC(kernel="linear")
lin_svm.fit(X_train_pca, y_train)

y_pred = lin_svm.predict(X_test_pca)
print("Linear SVM - accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

In [ ]:
%%time
# SVM with RBF kernel
rbf_svm = SVC(kernel="rbf", gamma="auto")
rbf_svm.fit(X_train_pca, y_train)
y_pred = rbf_svm.predict(X_test_pca)
print("SVM with RBF kernel - accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

### Parameter Tuning

### Exporting

In [ ]:
# Poly SVM Predictions 
predictions = []

for i in output_test_pca:
    prediction = poly_svm.predict([list(i)])
    predictions.append(prediction[0])

output_df = pd.DataFrame(predictions, columns = ['label'])
output_df.to_csv('./Output/polysvm.csv', sep=",", float_format='%d', index_label="id")